In [10]:
import os
import numpy as np
from PIL import Image
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, Activation
from tensorflow.keras.models import Sequential

In [11]:
# 数据集路径
data_dir = '/root/SKCNNs/256_ObjectCategories'

In [12]:
def preprocess(image, label):
    # 将图像尺寸调整为稍大的尺寸，然后随机裁剪到最终的 224x224 尺寸
    image = tf.image.resize(image, [256, 256])
    image = tf.image.random_crop(image, size=[224, 224, 3])
    # 可选：增加随机翻转
    image = tf.image.random_flip_left_right(image)
    # 归一化图像到 [0, 1] 范围
    image = image / 255.0
    return image, label

In [13]:
# 加载训练和验证数据集
batch_size = 16
img_height = 256
img_width = 256

train_data = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size
)

val_data = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size
)

Found 30607 files belonging to 257 classes.
Using 24486 files for training.
Found 30607 files belonging to 257 classes.
Using 6121 files for validation.


In [14]:
class SparseConv2D(layers.Layer):
    def __init__(self, filters, kernel_size, p, **kwargs):
        super(SparseConv2D, self).__init__(**kwargs)
        self.filters = filters
        self.kernel_size = kernel_size
        self.p = tf.Variable(float(p), trainable=False)  # 将 p 转换为浮点数类型
        self.counter = tf.Variable(0, trainable=False, dtype=tf.int32)  # 初始化计数器

    def build(self, input_shape):
        self.kernel = self.add_weight(name='kernel',
                                      shape=(self.kernel_size, self.kernel_size, input_shape[-1], self.filters),
                                      initializer='glorot_uniform',
                                      trainable=True)
        self.bias = self.add_weight(name='bias',
                                    shape=(self.filters,),
                                    initializer='zeros',
                                    trainable=True)

    @tf.function
    def call(self, inputs, training=None):
        if training:
            mask = tf.random.uniform(shape=(self.filters,), minval=0, maxval=1)
            mask = tf.cast(mask < self.p, dtype=tf.float32)
            mask = tf.reshape(mask, [1, 1, 1, self.filters])
            #self.counter.assign_add(1)  # 更新计数器
            #tf.print("\nP is", self.p)  # 使用 tf.print
        else:
            mask = tf.ones([1, 1, 1, self.filters], dtype=tf.float32) * self.p
    
        sparse_kernel = self.kernel * mask
        conv = tf.nn.conv2d(inputs, sparse_kernel, strides=[1, 1, 1, 1], padding='SAME')
        return tf.nn.bias_add(conv, self.bias)

    @tf.function
    def update_p(self, new_p):
        self.p.assign(float(new_p))  # 使用 assign 更新 tf.Variable 的值，并转换为浮点数
        #tf.print("\nEpoch counter is", self.counter)  # 使用 tf.print
        #tf.print("\nP is", self.p)  # 使用 tf.print

In [15]:
# 应用预处理，并设置批次大小和乱序缓冲区大小
train_data = train_data.map(lambda x, y: (tf.map_fn(lambda img: preprocess(img, y)[0], x, dtype=tf.float32), y)).shuffle(1000).prefetch(tf.data.experimental.AUTOTUNE)
val_data = val_data.map(lambda x, y: (tf.map_fn(lambda img: preprocess(img, y)[0], x, dtype=tf.float32), y)).prefetch(tf.data.experimental.AUTOTUNE)

In [16]:
# 使用较小的学习率和 Adam 优化器
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)

def alexnet_model(input_shape=(224, 224, 3), num_classes=257):
    model = Sequential([
        Conv2D(96, kernel_size=11, strides=4, padding='valid', activation='relu', input_shape=input_shape),
        BatchNormalization(),
        MaxPooling2D(pool_size=3, strides=2),
        Conv2D(256, kernel_size=5, padding='same', activation='relu'),
        BatchNormalization(),
        MaxPooling2D(pool_size=3, strides=2),
        Conv2D(384, kernel_size=3, padding='same', activation='relu'),
        Conv2D(384, kernel_size=3, padding='same', activation='relu'),
        SparseConv2D(filters=256, kernel_size=3, p=1, name='sparse_conv2d'),
        MaxPooling2D(pool_size=3, strides=2),
        Flatten(),
        Dense(4096, activation='relu'),
        Dropout(0.5),
        Dense(4096, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])
    return model

# 创建模型
model = alexnet_model()

# 编译模型
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 模型摘要
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 54, 54, 96)        34944     
                                                                 
 batch_normalization_2 (Batc  (None, 54, 54, 96)       384       
 hNormalization)                                                 
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 26, 26, 96)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 26, 26, 256)       614656    
                                                                 
 batch_normalization_3 (Batc  (None, 26, 26, 256)      1024      
 hNormalization)                                                 
                                                      

In [17]:
class UpdatePSparsity(tf.keras.callbacks.Callback):
    def __init__(self, model, sparsity_schedule):
        super(UpdatePSparsity, self).__init__()
        self.model = model
        self.sparsity_schedule = sparsity_schedule

    def on_epoch_end(self, epoch, logs=None):
        for layer_name, new_p in self.sparsity_schedule.items():
            layer = self.model.get_layer(name=layer_name)
            if epoch < len(new_p):
                p_value = new_p[epoch]
            else:
                p_value = new_p[-1]  # Use the last value for epochs beyond the predefined ones
            layer.update_p(p_value)
            #print(f"\nEpoch {epoch + 1}: Updated layer {layer_name} sparsity p to {p_value}")

sparsity_schedule = {
    'sparse_conv2d': [1]
}

In [18]:
#训练模型，并在验证集上验证
model.fit( train_data, epochs=40, validation_data=val_data, callbacks=[UpdatePSparsity(model, sparsity_schedule)] )

Epoch 1/40
1531/1531 [==============================] - 18s 9ms/step - loss: 5.0504 - accuracy: 0.0877 - val_loss: 4.5405 - val_accuracy: 0.1379
Epoch 2/40
1531/1531 [==============================] - 17s 9ms/step - loss: 4.4292 - accuracy: 0.1434 - val_loss: 4.3388 - val_accuracy: 0.1617
Epoch 3/40
1531/1531 [==============================] - 17s 8ms/step - loss: 3.9923 - accuracy: 0.2008 - val_loss: 3.8322 - val_accuracy: 0.2325
Epoch 4/40
1531/1531 [==============================] - 18s 9ms/step - loss: 3.6307 - accuracy: 0.2520 - val_loss: 3.8417 - val_accuracy: 0.2325
Epoch 5/40
1531/1531 [==============================] - 17s 8ms/step - loss: 3.3111 - accuracy: 0.3004 - val_loss: 3.5869 - val_accuracy: 0.2759
Epoch 6/40
1531/1531 [==============================] - 17s 9ms/step - loss: 3.0478 - accuracy: 0.3401 - val_loss: 3.3916 - val_accuracy: 0.3053
Epoch 7/40
1531/1531 [==============================] - 17s 9ms/step - loss: 2.8029 - accuracy: 0.3810 - val_loss: 3.0328 - val_ac